## Подготовка данных

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
df = pd.read_feather('allhabr.feather')

In [5]:
shit = ['2008-09','2013-05',
       '2013-04', '2012-12', '2011-03', '2012-10', '2011-11',
       '2010-10', '2009-05', '2010-01', '2008-10', '2008-11',
       '2008-12', '2009-01', '2009-02', '2009-03', '2009-04', '2009-06',
       '2009-07', '2009-08', '2009-09', '2009-11', '2009-12', '2010-02',
       '2010-03', '2010-04', '2010-05', '2010-11', '2011-01', '2011-02',
       '2011-04', '2011-05', '2011-06', '2011-12', '2012-01', '2012-02',
       '2012-03', '2012-04', '2012-05', '2012-07', '2012-08', '2012-09',
       '2012-11', '2013-01', '2013-02', '2013-03', '2013-06', '2013-07',
       '2013-08', '2013-11', '2010-08', '2012-06', '2013-09', '2013-10',
       '2008-08', '2009-10', '2010-09', '2011-07', '2011-08', '2011-09',
       '2011-10', '2007-07', '2007-08', '2007-09', '2007-10', '2008-01',
       '2008-04', '2008-06', '2010-06', '2010-12', '2006-07', '2006-08',
       '2006-09', '2006-10', '2006-11', '2006-12', '2007-01', '2007-02',
       '2007-04', '2007-05', '2007-06', '2007-11', '2007-12', '2008-02',
       '2008-03', '2008-05', '2008-07', '2007-03', '2010-07', '2013-12',
       '2006-06', '09:17-12', '09:09-12',
       '14:24-12', '11:11-12', '15:36-12']

In [6]:
df = df[df['date'].isin(set(df['date']) - set(shit))]

In [4]:
df['comments'] = df['comments'].astype('str')

In [ ]:
df['vot'] = df['votes'] / df['views']

### Нормальный код для Bag of words

In [127]:
data=[preprocess_text(text) for text in df['text'][:1000]]

In [138]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(analyzer = "word",   
                             tokenizer = None,    
                             preprocessor = None, 
                             stop_words = None,   
                             max_features = 5000) 

In [139]:
train_data_features = vectorizer.fit_transform(data)
train_data_features = train_data_features.toarray()
train_data_features

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [140]:
vocab = vectorizer.get_feature_names()
vocab

['aa',
 'ac',
 'accept',
 'access',
 'action',
 'active',
 'activerecord',
 'activity',
 'ad',
 'add',
 'addeventlistener',
 'addr',
 'address',
 'admin',
 'aes',
 'after',
 'ai',
 'ajax',
 'alert',
 'align',
 'all',
 'alloc',
 'allow',
 'alpha',
 'amazon',
 'amd',
 'amp',
 'an',
 'and',
 'android',
 'angular',
 'angularjs',
 'anim',
 'animal',
 'antlr',
 'any',
 'apache',
 'api',
 'apk',
 'app',
 'append',
 'apple',
 'application',
 'apply',
 'apt',
 'arduino',
 'are',
 'arg',
 'args',
 'arguments',
 'arm',
 'array',
 'as',
 'asm',
 'asp',
 'assignment',
 'ast',
 'asus',
 'at',
 'attr',
 'auth',
 'auto',
 'avg',
 'azure',
 'back',
 'background',
 'backup',
 'bar',
 'base',
 'bash',
 'be',
 'beeline',
 'before',
 'beg',
 'begin',
 'behaviorname',
 'bell',
 'better',
 'bh',
 'big',
 'bin',
 'binary',
 'bind',
 'binding',
 'bit',
 'bitcoin',
 'black',
 'block',
 'blockh',
 'blockw',
 'blog',
 'blue',
 'bluetooth',
 'bmp',
 'boaxxe',
 'body',
 'bold',
 'bool',
 'boolean',
 'boost',
 'boot

## Словарь тональности

In [12]:
dic = pd.read_csv('emo_dict.csv', delimiter=';')

In [253]:
dicdic = dic.set_index('term')['value'].to_dict()
dicdic['не'] = 'не'
known_words = set(dicdic.keys())

#### Подготовка текста

In [150]:
dat = df['comments'][0].split('~#')

In [ ]:
from nltk.corpus import stopwords
import nltk
import pymorphy2
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('stopwords')

In [176]:
def numb(word, df=dicdic):
    if word:
        return dicdic[word]
    return 0

In [279]:
def ne(comm):
    if 'не' in comm:
        comm[comm.index('не') + 1] *= -1
        comm.remove('не')
    return comm

In [281]:
def sentiment(comments, words=known_words):
    res = []
    new = [word_tokenize(comment.lower()) for comment in comments.split('~#')]
    comments = [[morph.parse(word)[0].normal_form  for word in sublist] for sublist in new]
    res = [list(map(numb, (set(comment) & known_words))) for comment in comments]
    res = list(map(ne, res))
    return res

In [283]:
list(map(sentiment, df['comments'][:2]))

[[[0.1, 0.3609, 0.4184, 0.6985, 0.2669, 0.4474],
  [0.514, 1.0, 0.6391, -0.9141, 0.25, 0.0, 0.0, 0.5787, 0.0, -0.9194],
  [0.0556,
   0.1,
   0.087,
   -0.2699,
   -0.2795,
   1.0,
   0.5066,
   0.0,
   0.0263,
   0.4023,
   0.0,
   0.1486,
   0.3761,
   -1.0,
   0.0,
   0.0,
   0.5196,
   0.1,
   0.25,
   0.1,
   0.5465,
   0.0,
   0.0,
   -0.9643,
   0.677,
   0.4706,
   1.0,
   1.0,
   0.5976,
   0.4141,
   0.0,
   1.0,
   0.25,
   0.4792,
   0.6724,
   0.25,
   0.0],
  []],
 [[0.5382, 0.0, 0.0],
  [],
  [0.1356, 0.436, 0.0, 0.381],
  [0.0, 0.6045, 0.0, 0.0, 0.5382],
  [0.0556,
   0.0,
   0.0347,
   0.9545,
   0.544,
   0.0455,
   0.4535,
   -0.1191,
   0.0806,
   0.3308,
   0.2669,
   0.5924,
   0.3761,
   0.0,
   0.0556,
   0.0,
   0.0,
   0.0,
   0.0,
   0.535,
   0.0556,
   0.4148,
   0.0556,
   -0.375,
   0.1111],
  [0.0, 1.0, 0.2179, -0.3761, 0.0556],
  [0.0357, 0.0926],
  [0.6493, 0.5476, 1.0, 0.3633, 0.1096, -0.0, 0.65, 0.8697],
  [0.6563, 0.0735],
  [0.7131,
   1.0,
   1.0,

* Составить список слов для хабов, тегов
* Подумать с поиском по тексту
* Подумать про вк/фэйсбук
* Придумать метрики
* Хабр Ку энд Эй